In [2]:
from csb_multi_env import *

2024-05-07 15:55:26,268	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [3]:
env = CodersStrikeBackMulti(dt=1)

In [4]:
env.reset()

({'car_0_0': array([ 3.00000000e+00, -3.13927058e+00,  1.19887241e+04,  4.74578865e+03,
          0.00000000e+00,  0.00000000e+00,  5.62000000e+03,  4.73100000e+03,
          7.92900000e+03,  3.07600000e+03,  7.37300000e+03,  5.76300000e+03,
          1.14840000e+04,  3.01800000e+03,  5.62000000e+03,  4.73100000e+03]),
  'car_1_0': array([3.00000000e+00, 2.95527997e+00, 1.16522414e+04, 3.59392955e+03,
         0.00000000e+00, 0.00000000e+00, 5.62000000e+03, 4.73100000e+03,
         7.92900000e+03, 3.07600000e+03, 7.37300000e+03, 5.76300000e+03,
         1.14840000e+04, 3.01800000e+03, 5.62000000e+03, 4.73100000e+03]),
  'car_0_1': array([3.00000000e+00, 2.75947902e+00, 1.13157586e+04, 2.44207045e+03,
         0.00000000e+00, 0.00000000e+00, 5.62000000e+03, 4.73100000e+03,
         7.92900000e+03, 3.07600000e+03, 7.37300000e+03, 5.76300000e+03,
         1.14840000e+04, 3.01800000e+03, 5.62000000e+03, 4.73100000e+03]),
  'car_1_1': array([3.00000000e+00, 2.57084427e+00, 1.09792759e+04, 1

In [5]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    done = done['__all__']
    rewards.append(reward)

In [6]:
rewards[-1]

{'car_0_0': 1, 'car_1_0': -1, 'car_0_1': 1, 'car_1_1': -1}

In [7]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [8]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-05-07 15:55:29,765	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.10.12
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8266


In [9]:
agent_ids = CodersStrikeBackMulti()._agent_ids
sym_policies = {agent_id: f"policy_{agent_ids[0]}" for agent_id in agent_ids}  # Symmetric
asym_policies = {agent_id: f"policy_{agent_id}" for agent_id in agent_ids}  # Asymmetric

def policy_mapping_fn(agent_id, episode, worker, *, policies=None, **kwargs):
    return policies[agent_id]


policies = sym_policies
policy_mapping = partial(policy_mapping_fn, policies=policies)

config = (
        PPOConfig()
        .environment(CodersStrikeBackMulti)
        .framework('torch')
        .training(train_batch_size=1024, gamma=0.99)
        .resources(num_gpus=0)
        .rollouts(num_rollout_workers=3)
        .multi_agent(policies=list(policies.values()),
                     policy_mapping_fn=policy_mapping)
    )

stop = {"timesteps_total": 12000}

# THE WAY OF CODING IS A BIT DIFFERENT HERE FROM THE SINGLE AGENT ONE.
tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(stop=stop),
    )

res = tuner.fit()
res.get_best_result(metric="loss", mode="min")
best_checkpoint = best_result.checkpoint


2024-05-07 15:55:32,036	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1822888) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1822888) 2024-05-07 15:55:39,668	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_con

NameError: name 'result_grid' is not defined

In [11]:
res.get_best_result(metric="loss", mode="min")
best_checkpoint = best_result.checkpoint

# Play against the above trained strategy

In [27]:
import random 

class CSBSelfPlay(MultiAgentEnv):
    def __init__(self):
        # These are all the possible agents (main can play as team 0 and 1.  Old can only play team 1)
        self.env = CodersStrikeBackMulti(dt=1)
        self._agent_ids = []
        for team in self.env.teams:
            for car in range(self.env.num_racers_per_team):
                self._agent_ids.append(f'main_{team}_{car}')
                if team > 0:
                    self._agent_ids.append(f'old_{team}_{car}')
        # Override default action space so that it covers all possible strategies
        self.observation_space = gym.spaces.Dict(make_agent_dictionary(self._agent_ids, self.env.ind_observation_space))
        self.action_space = gym.spaces.Dict(make_agent_dictionary(self._agent_ids, self.env.ind_action_space))
    
    def reset(self, **kwargs):
        self.create_competitor_pool()
        return self.env.reset(**kwargs)

    def create_competitor_pool(self):
        selected_ids = []
        policies = ['main'] + random.choices(['main','old'], weights=[0.8,0.2], k=self.env.num_teams-1)
        for team in range(self.env.num_teams):
            for car in range(self.env.num_racers_per_team):
                selected_ids.append(f'{policies[team]}_{team}_{car}')
        self.env._agent_ids = selected_ids
        for i, racer in enumerate(self.env.racers):
            racer.aid = selected_ids[i]

    def step(self, **kwargs):
        return self.env.step(**kwargs)

In [28]:
env = CSBSelfPlay()

In [29]:
env._agent_ids

['main_0_0', 'main_0_1', 'main_1_0', 'old_1_0', 'main_1_1', 'old_1_1']

In [40]:
env.reset()

({'main_0_0': array([ 3.00000000e+00, -2.72405712e+00,  9.81513763e+03,  1.01223473e+04,
          0.00000000e+00,  0.00000000e+00,  7.30000000e+02,  6.09200000e+03,
          1.19210000e+04,  6.66000000e+03,  1.02350000e+04,  8.37200000e+03,
          7.30000000e+02,  6.09200000e+03,  1.19210000e+04,  6.66000000e+03]),
  'main_0_1': array([ 3.00000000e+00, -2.84486058e+00,  1.00950459e+04,  8.95544909e+03,
          0.00000000e+00,  0.00000000e+00,  7.30000000e+02,  6.09200000e+03,
          1.19210000e+04,  6.66000000e+03,  1.02350000e+04,  8.37200000e+03,
          7.30000000e+02,  6.09200000e+03,  1.19210000e+04,  6.66000000e+03]),
  'old_1_0': array([ 3.00000000e+00, -2.96747351e+00,  1.03749541e+04,  7.78855091e+03,
          0.00000000e+00,  0.00000000e+00,  7.30000000e+02,  6.09200000e+03,
          1.19210000e+04,  6.66000000e+03,  1.02350000e+04,  8.37200000e+03,
          7.30000000e+02,  6.09200000e+03,  1.19210000e+04,  6.66000000e+03]),
  'old_1_1': array([ 3.00000000e+00